In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import re
 
# Read both Excel files into dataframes
epitope_df = pd.read_excel('JCV scored epitopes.xlsx')
strain_df = pd.read_excel('BKV strains_protein sequences.xlsx')
 
# Initialize a new dataframe to store the results
results_df = pd.DataFrame(columns=['Epitope', 'Gene', 'Strain'])
 
# Iterate over the protein sequences in the second file
for _, row in strain_df.iterrows():
    protein_sequence = row['Protein Sequence']
    gene = row['Gene Name']
    strain = row['Virus Strain Name']
 
    # Iterate over the epitopes in the first file
    for _, epitope in epitope_df.iterrows():
        epitope_name = epitope['Epitopes']
 
        # Use a regular expression to search for the epitope in the protein sequence
        if re.search(epitope_name, protein_sequence):
            results_df = results_df.append({'Epitope': epitope_name, 'Gene': gene, 'Strain': strain}, ignore_index=True)
 
# Export the results to a new Excel file
results_df.to_excel('results_JCV crossreactive epitopes.xlsx', index=False)

In [4]:
import pandas as pd
import seaborn as sns

# Read the data into a dataframe
results_df = pd.read_excel('results_JCV crossreactive epitopes.xlsx')

# Create a pivot table using the 'Epitope' and 'Strain' columns as the rows and columns of the table, and the count of the 'Epitope' column as the values of the table.
pivot_table = pd.pivot_table(results_df, 
                             index='Epitope', 
                             columns='Strain', 
                             values='Epitope', 
                             aggfunc=len,
                             fill_value=0)

# Print the pivot table
print(pivot_table)

# Use the to_excel method to save the pivot table to an excel sheet
pivot_table.to_excel('pivot_table_JCV crossreactive epitopes.xlsx', index=True)

Strain     AB211369.1  AB211370.1  AB211371.1  AB211372.1  AB211373.1  \
Epitope                                                                 
ARIPLPNL            1           1           1           1           1   
LLLIWFRPV           1           1           1           1           1   
SQHSTPPKK           1           1           1           1           1   
TPHRHRVSA           1           1           1           1           1   
VKNPYPISF           1           1           1           1           1   

Strain     AB211374.1  AB211375.1  AB211376.1  AB211377.1  AB211378.1  ...  \
Epitope                                                                ...   
ARIPLPNL            1           1           1           1           1  ...   
LLLIWFRPV           1           1           1           1           1  ...   
SQHSTPPKK           1           1           1           1           1  ...   
TPHRHRVSA           1           1           1           1           1  ...   
VKNPYPISF           

In [8]:
import pandas as pd

# Read input files
epitope_df = pd.read_excel('JCV scored epitopes.xlsx')
strain_df = pd.read_excel('BKV strains_protein sequences.xlsx')

# Remove empty epitopes
epitope_df = epitope_df.dropna(subset=['Epitopes'])

# Results dataframe
results_df = pd.DataFrame(columns=[
    'Epitope',
    'Matched_sequence',
    'Mismatches',
    'Gene',
    'Strain'
])

# Function to count mismatches
def count_mismatches(seq1, seq2):
    return sum(a != b for a, b in zip(seq1, seq2))

# Maximum mismatches allowed
MAX_MISMATCHES = 1  # change to 1 if you want to be stricter

# Loop through strains and epitopes
for _, row in strain_df.iterrows():
    protein_sequence = str(row['Protein sequence'])
    gene = row['Gene name']
    strain = row['Virus strain name']

    for _, ep in epitope_df.iterrows():
        epitope = str(ep['Epitopes'])
        ep_len = len(epitope)

        # Slide window across protein
        for i in range(len(protein_sequence) - ep_len + 1):
            window = protein_sequence[i:i + ep_len]
            mismatches = count_mismatches(epitope, window)

            if mismatches <= MAX_MISMATCHES:
                results_df = results_df.append({
                    'Epitope': epitope,
                    'Matched_sequence': window,
                    'Mismatches': mismatches,
                    'Gene': gene,
                    'Strain': strain
                }, ignore_index=True)

# Export results
results_df.to_excel('results_near_identity_crossreactive_epitopes_JCV.xlsx', index=False)

In [9]:
import pandas as pd

# Read input files
epitope_df = pd.read_excel('JCV scored epitopes.xlsx')
strain_df = pd.read_excel('BKV strains_protein sequences.xlsx')

# Remove empty epitopes
epitope_df = epitope_df.dropna(subset=['Epitopes'])

# Results dataframe
results_df = pd.DataFrame(columns=[
    'Epitope',
    'Matched_sequence',
    'Mismatches',
    'Gene',
    'Strain'
])

# Function to count mismatches
def count_mismatches(seq1, seq2):
    return sum(a != b for a, b in zip(seq1, seq2))

# Maximum mismatches allowed
MAX_MISMATCHES = 2  # change to 1 if you want to be stricter

# Loop through strains and epitopes
for _, row in strain_df.iterrows():
    protein_sequence = str(row['Protein sequence'])
    gene = row['Gene name']
    strain = row['Virus strain name']

    for _, ep in epitope_df.iterrows():
        epitope = str(ep['Epitopes'])
        ep_len = len(epitope)

        # Slide window across protein
        for i in range(len(protein_sequence) - ep_len + 1):
            window = protein_sequence[i:i + ep_len]
            mismatches = count_mismatches(epitope, window)

            if mismatches <= MAX_MISMATCHES:
                results_df = results_df.append({
                    'Epitope': epitope,
                    'Matched_sequence': window,
                    'Mismatches': mismatches,
                    'Gene': gene,
                    'Strain': strain
                }, ignore_index=True)

# Export results
results_df.to_excel('results_near_identity_crossreactive_epitopes_JCV_2.xlsx', index=False)

In [10]:
import pandas as pd

# =========================
# Read near-identity results
# =========================
results_df = pd.read_excel(
    'results_near_identity_crossreactive_epitopes_JCV.xlsx'
)

# =========================
# 1. Pivot table (counts)
# =========================
pivot_table = pd.pivot_table(
    results_df,
    index='Epitope',
    columns='Strain',
    values='Matched_sequence',
    aggfunc='count',
    fill_value=0
)

# ==========================================
# 2. Strain-resolved sequence summary table
# ==========================================
sequence_summary = (
    results_df
    .groupby(['Epitope', 'Strain'])
    .agg({
        'Matched_sequence': lambda x: ', '.join(sorted(set(x))),
        'Mismatches': 'min'
    })
    .reset_index()
)

# ==========================================
# 3. Strain coverage per epitope
# ==========================================
strain_coverage = (
    sequence_summary
    .groupby('Epitope')['Strain']
    .nunique()
    .reset_index(name='Number_of_JCV_strains')
)

# =========================
# Save everything together
# =========================
output_file = 'near_identity_crossreactivity_JCV_summary.xlsx'

with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    pivot_table.to_excel(
        writer,
        sheet_name='Pivot_Epitope_by_Strain'
    )
    sequence_summary.to_excel(
        writer,
        sheet_name='Sequences_per_Strain',
        index=False
    )
    strain_coverage.to_excel(
        writer,
        sheet_name='Strain_Coverage',
        index=False
    )

print(f"All results saved to {output_file}")

All results saved to near_identity_crossreactivity_JCV_summary.xlsx


In [11]:
import pandas as pd

# =========================
# Read near-identity results
# =========================
results_df = pd.read_excel(
    'results_near_identity_crossreactive_epitopes_JCV_2.xlsx'
)

# =========================
# 1. Pivot table (counts)
# =========================
pivot_table = pd.pivot_table(
    results_df,
    index='Epitope',
    columns='Strain',
    values='Matched_sequence',
    aggfunc='count',
    fill_value=0
)

# ==========================================
# 2. Strain-resolved sequence summary table
# ==========================================
sequence_summary = (
    results_df
    .groupby(['Epitope', 'Strain'])
    .agg({
        'Matched_sequence': lambda x: ', '.join(sorted(set(x))),
        'Mismatches': 'min'
    })
    .reset_index()
)

# ==========================================
# 3. Strain coverage per epitope
# ==========================================
strain_coverage = (
    sequence_summary
    .groupby('Epitope')['Strain']
    .nunique()
    .reset_index(name='Number_of_JCV_strains')
)

# =========================
# Save everything together
# =========================
output_file = 'near_identity_crossreactivity_summary_JCV_2.xlsx'

with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    pivot_table.to_excel(
        writer,
        sheet_name='Pivot_Epitope_by_Strain'
    )
    sequence_summary.to_excel(
        writer,
        sheet_name='Sequences_per_Strain',
        index=False
    )
    strain_coverage.to_excel(
        writer,
        sheet_name='Strain_Coverage',
        index=False
    )

print(f"All results saved to {output_file}")

All results saved to near_identity_crossreactivity_summary_JCV_2.xlsx
